In [ ]:
!pip install numpy pandas scikit-learn tqdm scipy openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import scipy as sp
from concurrent.futures import ThreadPoolExecutor

In [ ]:
! pip install -U LibRecommender

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
books = pd.read_csv('/content/drive/MyDrive/Books_Extended.csv', delimiter=';', low_memory=False)
ratings = pd.read_csv('/content/drive/MyDrive/Ratings.csv', delimiter=';')
users = pd.read_csv('/content/drive/MyDrive/Users.csv', delimiter=';')

<ipython-input-6-965afd468f24>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('/content/drive/MyDrive/Users.csv', delimiter=';')


## Data Preparation

In [ ]:
df_prep_step_1 = pd.merge(books, ratings.query("`Rating` > 0"), on='ISBN', how='inner')

df_prep_step_2 = pd.merge(df_prep_step_1, users, on='User-ID', how='inner')

In [ ]:
df_prep = df_prep_step_2.drop(['Publisher'], axis=1)
df_isbn = df_prep.drop_duplicates()

In [ ]:
df_prep = df_prep_step_2.drop(['Publisher'], axis=1)

In [ ]:
df_isbn['Original_NaN'] = df_isbn['Age'].isna()

df_isbn['Age'] = pd.to_numeric(df_isbn['Age'], errors='coerce')

df_isbn = df_isbn[~(users['Age'].isna() & ~df_isbn['Original_NaN'])]

df_isbn.drop(columns=['Original_NaN'], inplace=True)

<ipython-input-10-a229782017b2>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_isbn = df_isbn[~(users['Age'].isna() & ~df_isbn['Original_NaN'])]
<ipython-input-10-a229782017b2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isbn.drop(columns=['Original_NaN'], inplace=True)


In [ ]:
df_isbn['User-ID'] = pd.to_numeric(df_isbn['User-ID'], errors='coerce')

df_isbn = df_isbn[~(df_isbn['User-ID'].isna())]

In [ ]:
age_outliers = df_isbn.query("Age > 100 or Age < 6")

user_outliers = age_outliers["User-ID"].to_list()

In [ ]:
# exclude age outliers
df_isbn = df_isbn[~df_isbn["User-ID"].isin(user_outliers)]

## Fetching Embeddings

In [ ]:
unique_descriptions = df_isbn['Description'].unique()

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='key')

def get_embedding(text, model="text-embedding-3-small"):
    if text is None or pd.isna(text):
        return None
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [ ]:
description_to_embedding = {}

for description in tqdm(unique_descriptions, desc="Generating Embeddings"):
    if description not in description_to_embedding:
        embedding = get_embedding(description)
        description_to_embedding[description] = embedding

# Now, map the embeddings back to the original DataFrame
df_isbn['embedding'] = df_isbn['Description'].map(description_to_embedding)


Generating Embeddings: 100%|██████████| 38475/38475 [2:16:05<00:00,  4.71it/s]


In [ ]:
df_isbn['embedding'].notna().sum()

79833

In [ ]:
df_cleaned.head()

,item,Title,Author,Year,Description,user,label,Age,embedding,Lang_cat,...,pca_dim_1,pca_dim_2,pca_dim_3,pca_dim_4,pca_dim_5,pca_dim_6,pca_dim_7,pca_dim_8,pca_dim_9,pca_dim_10
6,1881320189,Goodbye to the Buttermilk Sky,Julia Oliver,1994.0,This highly praised first novel by fiction wri...,8.0,7.0,NaN,"[0.02566627413034439, 0.014639468863606453, -0...",False,...,-0.049454,-0.031209,0.024619,-0.021410,-0.007651,0.053179,0.011405,-0.063071,0.005837,0.101811
17,0449005615,Seabiscuit: An American Legend,LAURA HILLENBRAND,2002.0,<b>There's an alternate cover edition <a href=...,11676.0,10.0,NaN,"[0.015707721933722496, 0.013525345362722874, -...",False,...,-0.048156,-0.050934,0.024829,0.188009,0.005003,0.033236,-0.016880,0.005270,-0.003529,0.006175
21,042518630X,Purity in Death,J.D. Robb,2002.0,Louie Cogburn had spent three days holed up in...,11676.0,10.0,NaN,"[-0.0024547278881073, 0.02609311230480671, -0....",False,...,-0.129184,-0.020992,-0.094960,0.015528,-0.004632,-0.024696,0.057121,0.009259,0.002623,0.021764
24,042511774X,Breathing Lessons,Anne Tyler,1994.0,Maggie Moran's mission is to connect and unite...,11676.0,5.0,NaN,"[0.007486206945031881, 0.0089639313519001, -0....",False,...,0.019269,0.027044,0.072336,0.013097,-0.081529,-0.031606,-0.063961,-0.027216,-0.012653,-0.010802
28,0312261594,Female Intelligence,Jane Heller,2001.0,"In a modern twist on My Fair Lady, Lynn Wyman ...",11676.0,6.0,NaN,"[7.745592824903724e-07, 0.021870341151952744, ...",False,...,0.147074,0.029381,-0.098413,0.003312,0.000773,-0.056226,-0.082996,-0.063480,-0.047610,-0.033220


We save only data that has embeddings, and then reduce dimentionality using PCA

In [ ]:
df_cleaned = df_isbn.dropna(subset=['embedding'])

In [ ]:
df_cleaned['Language'] = df_cleaned['Language'].fillna('missing')

<ipython-input-143-df92cbe22f08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Language'] = df_cleaned['Language'].fillna('missing')


In [ ]:
df_cleaned.rename(columns={"User-ID": "user"}, inplace=True)

<ipython-input-144-939667d10c5d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={"User-ID": "user"}, inplace=True)


In [ ]:
df_cleaned.rename(columns={"ISBN": "item"}, inplace=True)

<ipython-input-145-24d637dee9f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={"ISBN": "item"}, inplace=True)


In [ ]:
df_cleaned.rename(columns={"Rating": "label"}, inplace=True)

<ipython-input-146-21c57fff3936>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={"Rating": "label"}, inplace=True)


In [ ]:
# Perform one-hot encoding on the Language column
df_cleaned = pd.get_dummies(df_cleaned, columns=['Language'], prefix='Lang')

In [ ]:
dummy_columns = [col for col in df_cleaned.columns if col.startswith('Lang')]

In [ ]:
pca_columns = [f'pca_dim_{i+1}' for i in range(10)]

In [ ]:
columns = ['user', 'item', 'label']
user_cols = ['Age']
item_cols = pca_columns + ['Year']

columns = columns + user_cols + item_cols

In [ ]:
columns

['user',
 'item',
 'label',
 'Age',
 'pca_dim_1',
 'pca_dim_2',
 'pca_dim_3',
 'pca_dim_4',
 'pca_dim_5',
 'pca_dim_6',
 'pca_dim_7',
 'pca_dim_8',
 'pca_dim_9',
 'pca_dim_10',
 'Year']

In [ ]:
df_cleaned_dropped = df_cleaned.dropna()

In [ ]:
df_cleaned['embedding_256'] = df_cleaned['embedding'].apply(lambda x: x[:256])

In [ ]:
import numpy as np

# Convert the embeddings to a 2D NumPy array
embeddings_256 = np.array(df_cleaned['embedding_256'].tolist())

In [ ]:
from sklearn.decomposition import PCA

# Apply PCA to reduce to 10 dimensions
pca = PCA(n_components=10)
reduced_embeddings_10 = pca.fit_transform(embeddings_256)


In [ ]:
# Create column names for the new dimensions
pca_columns = [f'pca_dim_{i+1}' for i in range(10)]

# Convert the reduced embeddings to a DataFrame
pca_df = pd.DataFrame(reduced_embeddings_10, columns=pca_columns)

# Concatenate the new PCA columns to the original DataFrame
df_cleaned = pd.concat([df_cleaned_dropped, pca_df], axis=1)

In [ ]:
df_cleaned_dropped[columns].to_csv('df_with_emb_cleaned.csv', index=False)

In [ ]:
from libreco.algorithms import ALS
from sklearn.model_selection import train_test_split

from libreco.data import DataInfo, DatasetFeat, split_by_ratio
train_df, test_df = train_test_split(
    df_cleaned_dropped[columns],
    test_size=0.3,
    random_state=42
)

eval_user_mask = test_df['user'].isin(train_df['user'].unique())
eval_item_mask = test_df['item'].isin(train_df['item'].unique())

eval_df = test_df[eval_user_mask & eval_item_mask]

train_data, data_info = DatasetFeat.build_trainset(train_df, user_col=user_cols, item_col=item_cols, dense_col=item_cols + user_cols)
test_data = DatasetFeat.build_testset(eval_df)

from libreco.algorithms import ALS
from sklearn.model_selection import train_test_split

from libreco.data import DataInfo, DatasetFeat, split_by_ratio
train_df, test_df = train_test_split(
    df_cleaned_dropped[columns],
    test_size=0.3,
    random_state=42
)

eval_user_mask = test_df['user'].isin(train_df['user'].unique())
eval_item_mask = test_df['item'].isin(train_df['item'].unique())

eval_df = test_df[eval_user_mask & eval_item_mask]

train_data, data_info = DatasetFeat.build_trainset(train_df, user_col=user_cols, item_col=item_cols, dense_col=item_cols + user_cols)
test_data = DatasetFeat.build_testset(eval_df)

model = ALS(
    task="rating",           # Specifies that this is a rating prediction task
    data_info=data_info,     # Provides dataset information
    embed_size=128,           # Size of the latent embeddings
    n_epochs=2000,             # Number of training epochs
    reg=0.1,                # Regularization parameter
    alpha=10,               # Confidence parameter (higher values increase the influence of observed data)
    use_cg=False,             # Use Conjugate Gradient method
    n_threads=4,             # Number of threads to use
    seed=42                  # Random seed for reproducibility
)


model.fit(train_data, verbose=2, neg_sampling=False, eval_data=test_data, metrics=['rmse','mae'])

Training start time: 2024-08-11 11:55:44
Epoch 1 elapsed: 1.219s


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 424.65it/s]

	 eval rmse: 7.0310
	 eval mae: 6.8017


Epoch 2 elapsed: 1.173s


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 148.96it/s]

...

eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 927.74it/s]

	 eval rmse: 5.4495
	 eval mae: 4.7998


Epoch 2000 elapsed: 0.962s


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 960.67it/s]

	 eval rmse: 5.4494
	 eval mae: 4.7997


In [ ]:
predictions = {}

In [ ]:
prediction = {}
for user_id in tqdm(eval_df['user'].unique()):
    predictions[user_id] = model.recommend_user(user=user_id, n_rec=20)[user_id]

100%|██████████| 273/273 [00:00<00:00, 290.64it/s]


In [ ]:
mean_ap = 0
total_abs_loss = 0
total_l2_loss = 0
total_precision_at_k = 0
total_recall_at_k = 0
total_ndcg = 0
total_dcg = 0

count_users_with_relevant_items = 0
count = 0

for user_id in eval_df['user'].unique():
    relevant_items = eval_df[eval_df['user']==user_id]['item'].to_numpy()

    predicted_isbns = predictions.get(user_id, [])

    if predicted_isbns is None:
        continue

    ap = average_precision(predicted_isbns, relevant_items, m=20)
    mean_ap += ap

    precision = precision_at_k(predicted_isbns, relevant_items, k=20)
    recall = recall_at_k(predicted_isbns, relevant_items, k=20)
    total_precision_at_k += precision
    total_recall_at_k += recall

    count_users_with_relevant_items += 1

mean_ap = mean_ap / count_users_with_relevant_items if count_users_with_relevant_items > 0 else 0
mean_precision_at_k = total_precision_at_k / count_users_with_relevant_items if count_users_with_relevant_items > 0 else 0
mean_recall_at_k = total_recall_at_k / count_users_with_relevant_items if count_users_with_relevant_items > 0 else 0

print(f"Mean Average Precision: {mean_ap}")
print(f"Mean Precision at K: {mean_precision_at_k}")
print(f"Mean Recall at K: {mean_recall_at_k}")

Mean Average Precision: 0.00490744374672946
Mean Precision at K: 0.001098901098901099
Mean Recall at K: 0.01380167451596023
